In [1]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd
from config import DEVICE, SEED, MODEL_CONFIG, TRAINING_CONFIG, DATASET_CONFIG
from model import JointCausalModel
from utility import compute_class_weights, label_value_counts
from dataset_collator import CausalDataset, CausalDatasetCollator
from config import id2label_cls, id2label_bio, id2label_rel
from evaluate_joint_causal_model import evaluate_model, print_eval_report
from trainer import train_model
import random
from transformers import AutoTokenizer

/home/rnorouzini/JointLearning/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_data_path = "/home/rnorouzini/JointLearning/datasets/expert_multi_task_data/train.csv"
val_data_path = "/home/rnorouzini/JointLearning/datasets/expert_multi_task_data/val.csv"
test_data_path = "/home/rnorouzini/JointLearning/datasets/expert_multi_task_data/test.csv"
train_df = pd.read_csv(train_data_path)
val_df = pd.read_csv(val_data_path)
test_df = pd.read_csv(test_data_path)

In [3]:
train_dataset = CausalDataset(
    train_df,
    tokenizer_name=MODEL_CONFIG["encoder_name"],
    max_length=DATASET_CONFIG["max_length"],
)
# %%
val_dataset = CausalDataset(
    val_df,
    tokenizer_name=MODEL_CONFIG["encoder_name"],
    max_length=DATASET_CONFIG["max_length"],
)
# %%
test_dataset = CausalDataset(
    test_df,
    tokenizer_name=MODEL_CONFIG["encoder_name"],
    max_length=DATASET_CONFIG["max_length"],
)
# %%

In [4]:
labels_flat = label_value_counts(train_dataset)
# %%
cls_label_flat = labels_flat["cls_labels_flat"]
bio_label_flat = labels_flat["bio_labels_flat"]
rel_label_flat = labels_flat["rel_labels_flat"]
# %%
# Calculate class weights
cls_weights = compute_class_weights(labels_list=cls_label_flat, num_classes=MODEL_CONFIG["num_cls_labels"], technique="ens", ignore_index=-100)
bio_weights = compute_class_weights(labels_list=bio_label_flat, num_classes=MODEL_CONFIG["num_bio_labels"], technique="ens", ignore_index=-100)
rel_weights = compute_class_weights(labels_list=rel_label_flat, num_classes=MODEL_CONFIG["num_rel_labels"], technique="ens", ignore_index=-100)
print(f"CLS Weights: {cls_weights}")
print(f"BIO Weights: {bio_weights}")
print(f"REL Weights: {rel_weights}")
# %%

cls_labels_value_counts:
 0    1047
1    1035
Name: count, dtype: int64
bio_labels_value_counts:
  6      52764
 3       8717
 1       6948
-100     4164
 2       1320
 0       1179
 5        483
 4         79
Name: count, dtype: int64
rel_labels_value_counts:
 0    2887
1    1494
Name: count, dtype: int64
CLS Weights: tensor([0.0015, 0.0016])
BIO Weights: tensor([0.0014, 0.0010, 0.0014, 0.0010, 0.0132, 0.0026, 0.0010])
REL Weights: tensor([0.0011, 0.0013])


In [5]:
collator = CausalDatasetCollator(
    tokenizer=train_dataset.tokenizer
)
# %%
# take a 100 samples from train_dataset
# train_dataset = torch.utils.data.Subset(train_dataset, random.sample(range(len(train_dataset)), 20))
# val_dataset = torch.utils.data.Subset(val_dataset, random.sample(range(len(val_dataset)), 20))
# # %%
train_dataloader = DataLoader(
    train_dataset,
    batch_size=TRAINING_CONFIG["batch_size"],
    collate_fn=collator,
    shuffle=True

)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=TRAINING_CONFIG["batch_size"],
    collate_fn=collator,
    shuffle=False
)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_CONFIG["encoder_name"])
model_path = r"/home/rnorouzini/JointLearning/src/jointlearning/expert_bert_GCE_weakSP/expert_bert_GCE_weakSP_model.pt"
model = JointCausalModel(**MODEL_CONFIG)
model.load_state_dict(torch.load(model_path, map_location=DEVICE))
model.to(DEVICE)
model.eval()

JointCausalModel(
  (enc): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [7]:
optimizer = optim.AdamW(
    model.parameters(),
    lr=TRAINING_CONFIG["learning_rate"],
    weight_decay=TRAINING_CONFIG["weight_decay"]
)
# %%
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.1,
    patience=2
)

In [9]:
model_save_path = r"/home/rnorouzini/JointLearning/src/jointlearning/expert_bert_GCE_Softmax_Normal/expert_bert_GCE_Softmax_Normal_model.pt"

In [10]:
trained_model, training_history = train_model(
        model=model,
        train_dataloader=train_dataloader,
        val_dataloader=val_dataloader,
        optimizer=optimizer,
        num_epochs=TRAINING_CONFIG["num_epochs"],
        device=DEVICE,
        id2label_cls=id2label_cls,
        id2label_bio=id2label_bio,
        id2label_rel=id2label_rel,
        model_save_path=model_save_path,
        scheduler=scheduler,
        cls_class_weights=cls_weights,
        bio_class_weights=bio_weights, # Only for softmax
        rel_class_weights=rel_weights,
        patience_epochs=TRAINING_CONFIG["num_epochs"],
        seed=SEED,
        task_loss_weights={"cls": 1.0, "bio": 4.0, "rel": 1.0},
        max_grad_norm=TRAINING_CONFIG["gradient_clip_val"],
        eval_fn_metrics=evaluate_model, # Pass your evaluate_model function here
        print_report_fn=print_eval_report # Pass your print_eval_report function here
    )

--- Training Configuration ---
Device: cuda
Number of Epochs: 20
Seed: 8642
Optimizer: AdamW (LR: 1e-05, Weight Decay: 0.1)
Scheduler: ReduceLROnPlateau
Gradient Clipping: Enabled (Max Norm: 1.0)
Early Stopping Patience: 20
Model Save Path: /home/rnorouzini/JointLearning/src/jointlearning/expert_bert_GCE_Softmax_Normal/expert_bert_GCE_Softmax_Normal_model.pt
Mode: Standard Training (CrossEntropy)
Using task loss weights: {'cls': 1.0, 'bio': 4.0, 'rel': 1.0}
CLS Class Weights: Provided
BIO Class Weights: Provided
REL Class Weights: Provided
----------------------------


Epoch 1/20 [Training]:   0%|          | 0/131 [00:00<?, ?it/s]You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



Epoch 1/20 Summary
--------------------------------------------------------------------------------
  Average Training Loss:           4.5763
  Average Validation Loss:         1.2018
  Overall Validation Avg F1 (Macro): 0.7581
--------------------------------------------------------------------------------
Task-Specific Validation Performance:

  [Task 1: Sentence Classification]
    Macro F1-Score:  0.8166
    Macro Precision: 0.8298
    Macro Recall:    0.8195
    Accuracy:        0.8178
    Per-class details:
      non-causal  : F1=0.8019 (P=0.8973, R=0.7249, Support=229.0)
      causal      : F1=0.8313 (P=0.7623, R=0.9140, Support=221.0)

  [Task 2: BIO Prediction (Token-BIO)]
    Macro F1-Score:  0.5296
    Macro Precision: 0.4934
    Macro Recall:    0.6261
    Per-tag details (P=Precision, R=Recall, F1=F1-Score, S=Support):
      B-C       : F1=0.484 (P=0.443, R=0.532, S=263.0)
      I-C       : F1=0.598 (P=0.629, R=0.570, S=1451.0)
      B-E       : F1=0.413 (P=0.376, R=0.458


Epoch 2/20 Summary
--------------------------------------------------------------------------------
  Average Training Loss:           2.5244
  Average Validation Loss:         1.1478
  Overall Validation Avg F1 (Macro): 0.7582
--------------------------------------------------------------------------------
Task-Specific Validation Performance:

  [Task 1: Sentence Classification]
    Macro F1-Score:  0.8254
    Macro Precision: 0.8402
    Macro Recall:    0.8284
    Accuracy:        0.8267
    Per-class details:
      non-causal  : F1=0.8107 (P=0.9126, R=0.7293, Support=229.0)
      causal      : F1=0.8402 (P=0.7678, R=0.9276, Support=221.0)

  [Task 2: BIO Prediction (Token-BIO)]
    Macro F1-Score:  0.5312
    Macro Precision: 0.4835
    Macro Recall:    0.6748
    Per-tag details (P=Precision, R=Recall, F1=F1-Score, S=Support):
      B-C       : F1=0.504 (P=0.449, R=0.574, S=263.0)
      I-C       : F1=0.646 (P=0.619, R=0.676, S=1451.0)
      B-E       : F1=0.436 (P=0.395, R=0.487


Epoch 3/20 Summary
--------------------------------------------------------------------------------
  Average Training Loss:           1.9657
  Average Validation Loss:         1.2024
  Overall Validation Avg F1 (Macro): 0.7720
--------------------------------------------------------------------------------
Task-Specific Validation Performance:

  [Task 1: Sentence Classification]
    Macro F1-Score:  0.8350
    Macro Precision: 0.8428
    Macro Recall:    0.8368
    Accuracy:        0.8356
    Per-class details:
      non-causal  : F1=0.8255 (P=0.8974, R=0.7642, Support=229.0)
      causal      : F1=0.8445 (P=0.7882, R=0.9095, Support=221.0)

  [Task 2: BIO Prediction (Token-BIO)]
    Macro F1-Score:  0.5399
    Macro Precision: 0.5040
    Macro Recall:    0.6294
    Per-tag details (P=Precision, R=Recall, F1=F1-Score, S=Support):
      B-C       : F1=0.512 (P=0.480, R=0.548, S=263.0)
      I-C       : F1=0.648 (P=0.655, R=0.641, S=1451.0)
      B-E       : F1=0.435 (P=0.403, R=0.472

...


Epoch 16/20 Summary
--------------------------------------------------------------------------------
  Average Training Loss:           1.1131
  Average Validation Loss:         1.4085
  Overall Validation Avg F1 (Macro): 0.7724
--------------------------------------------------------------------------------
Task-Specific Validation Performance:

  [Task 1: Sentence Classification]
    Macro F1-Score:  0.8371
    Macro Precision: 0.8465
    Macro Recall:    0.8392
    Accuracy:        0.8378
    Per-class details:
      non-causal  : F1=0.8266 (P=0.9062, R=0.7598, Support=229.0)
      causal      : F1=0.8476 (P=0.7868, R=0.9186, Support=221.0)

  [Task 2: BIO Prediction (Token-BIO)]
    Macro F1-Score:  0.5361
    Macro Precision: 0.4933
    Macro Recall:    0.6315
    Per-tag details (P=Precision, R=Recall, F1=F1-Score, S=Support):
      B-C       : F1=0.513 (P=0.463, R=0.574, S=263.0)
      I-C       : F1=0.654 (P=0.634, R=0.676, S=1451.0)
      B-E       : F1=0.445 (P=0.410, R=0.48


Epoch 17/20 Summary
--------------------------------------------------------------------------------
  Average Training Loss:           1.0743
  Average Validation Loss:         1.4502
  Overall Validation Avg F1 (Macro): 0.7728
--------------------------------------------------------------------------------
Task-Specific Validation Performance:

  [Task 1: Sentence Classification]
    Macro F1-Score:  0.8371
    Macro Precision: 0.8465
    Macro Recall:    0.8392
    Accuracy:        0.8378
    Per-class details:
      non-causal  : F1=0.8266 (P=0.9062, R=0.7598, Support=229.0)
      causal      : F1=0.8476 (P=0.7868, R=0.9186, Support=221.0)

  [Task 2: BIO Prediction (Token-BIO)]
    Macro F1-Score:  0.5361
    Macro Precision: 0.4933
    Macro Recall:    0.6315
    Per-tag details (P=Precision, R=Recall, F1=F1-Score, S=Support):
      B-C       : F1=0.513 (P=0.463, R=0.574, S=263.0)
      I-C       : F1=0.654 (P=0.634, R=0.676, S=1451.0)
      B-E       : F1=0.445 (P=0.410, R=0.48


Epoch 18/20 Summary
--------------------------------------------------------------------------------
  Average Training Loss:           1.0956
  Average Validation Loss:         1.4401
  Overall Validation Avg F1 (Macro): 0.7749
--------------------------------------------------------------------------------
Task-Specific Validation Performance:

  [Task 1: Sentence Classification]
    Macro F1-Score:  0.8371
    Macro Precision: 0.8465
    Macro Recall:    0.8392
    Accuracy:        0.8378
    Per-class details:
      non-causal  : F1=0.8266 (P=0.9062, R=0.7598, Support=229.0)
      causal      : F1=0.8476 (P=0.7868, R=0.9186, Support=221.0)

  [Task 2: BIO Prediction (Token-BIO)]
    Macro F1-Score:  0.5353
    Macro Precision: 0.4930
    Macro Recall:    0.6295
    Per-tag details (P=Precision, R=Recall, F1=F1-Score, S=Support):
      B-C       : F1=0.514 (P=0.465, R=0.574, S=263.0)
      I-C       : F1=0.654 (P=0.634, R=0.675, S=1451.0)
      B-E       : F1=0.446 (P=0.411, R=0.48


Epoch 19/20 Summary
--------------------------------------------------------------------------------
  Average Training Loss:           1.0779
  Average Validation Loss:         1.4068
  Overall Validation Avg F1 (Macro): 0.7749
--------------------------------------------------------------------------------
Task-Specific Validation Performance:

  [Task 1: Sentence Classification]
    Macro F1-Score:  0.8371
    Macro Precision: 0.8465
    Macro Recall:    0.8392
    Accuracy:        0.8378
    Per-class details:
      non-causal  : F1=0.8266 (P=0.9062, R=0.7598, Support=229.0)
      causal      : F1=0.8476 (P=0.7868, R=0.9186, Support=221.0)

  [Task 2: BIO Prediction (Token-BIO)]
    Macro F1-Score:  0.5353
    Macro Precision: 0.4930
    Macro Recall:    0.6295
    Per-tag details (P=Precision, R=Recall, F1=F1-Score, S=Support):
      B-C       : F1=0.514 (P=0.465, R=0.574, S=263.0)
      I-C       : F1=0.654 (P=0.634, R=0.675, S=1451.0)
      B-E       : F1=0.446 (P=0.411, R=0.48


Epoch 20/20 Summary
--------------------------------------------------------------------------------
  Average Training Loss:           1.0880
  Average Validation Loss:         1.4109
  Overall Validation Avg F1 (Macro): 0.7721
--------------------------------------------------------------------------------
Task-Specific Validation Performance:

  [Task 1: Sentence Classification]
    Macro F1-Score:  0.8371
    Macro Precision: 0.8465
    Macro Recall:    0.8392
    Accuracy:        0.8378
    Per-class details:
      non-causal  : F1=0.8266 (P=0.9062, R=0.7598, Support=229.0)
      causal      : F1=0.8476 (P=0.7868, R=0.9186, Support=221.0)

  [Task 2: BIO Prediction (Token-BIO)]
    Macro F1-Score:  0.5353
    Macro Precision: 0.4930
    Macro Recall:    0.6295
    Per-tag details (P=Precision, R=Recall, F1=F1-Score, S=Support):
      B-C       : F1=0.514 (P=0.465, R=0.574, S=263.0)
      I-C       : F1=0.654 (P=0.634, R=0.675, S=1451.0)
      B-E       : F1=0.446 (P=0.411, R=0.48

In [11]:
trained_model.save_pretrained(
    r"/home/rnorouzini/JointLearning/src/jointlearning/expert_bert_GCE_Softmax_Normal/hf_exper_bert_GCE_Softmax_Normal"
)

In [12]:
train_dataset.tokenizer.save_pretrained(
    r"/home/rnorouzini/JointLearning/src/jointlearning/expert_bert_GCE_Softmax_Normal/hf_exper_bert_GCE_Softmax_Normal"
)

('/home/rnorouzini/JointLearning/src/jointlearning/expert_bert_GCE_Softmax_Normal/hf_exper_bert_GCE_Softmax_Normal/tokenizer_config.json',
 '/home/rnorouzini/JointLearning/src/jointlearning/expert_bert_GCE_Softmax_Normal/hf_exper_bert_GCE_Softmax_Normal/special_tokens_map.json',
 '/home/rnorouzini/JointLearning/src/jointlearning/expert_bert_GCE_Softmax_Normal/hf_exper_bert_GCE_Softmax_Normal/vocab.txt',
 '/home/rnorouzini/JointLearning/src/jointlearning/expert_bert_GCE_Softmax_Normal/hf_exper_bert_GCE_Softmax_Normal/added_tokens.json',
 '/home/rnorouzini/JointLearning/src/jointlearning/expert_bert_GCE_Softmax_Normal/hf_exper_bert_GCE_Softmax_Normal/tokenizer.json')